import pickle
# ^^^ pyforest auto-imports - don't write above this line
**EDA for NHL Game Data**

# Imports 

In [8]:
from functions import *

# Getting General Info about database

## Tables

In [9]:
test_query = "SHOW TABLES;"
all_tables_query_result = run_query(test_query)
all_tables = [a[0] for a in all_tables_query_result]
all_tables # ignoring entries table

['entries',
 'game',
 'game_goalie_stats',
 'game_plays',
 'game_plays_players',
 'game_shifts',
 'game_skater_stats',
 'game_teams_stats',
 'player_info',
 'team_info']

## Dataframe with info about every column in every table

Making all of the columns and their datatypes into a table so that it will be easier later to find out how to join tables.

In [10]:
# each table is stored in a list with each of the columns as values to the table key
all_query_results = []
for table in all_tables[1:]:
    query = f"""
                SHOW columns FROM {table}
            """
    query_result = run_query(query) # returns:  Field       | Type     | Null | Key | Default | Extra     
    table_w_query = {table: query_result}
    all_query_results.append(table_w_query)

In [11]:
# get all columns in a contingency table (table name will be the last value in each sublist)
all_columns = [] 
for thing in all_query_results:
    for key, value in thing.items():
        for item in value:
            list_item = list(item)
            list_item.append(key)
            all_columns.append(list_item)

In [12]:
# removing incorrect values
for x in all_columns:
    if len(x) != 7:
        all_columns.remove(x)

In [13]:
table_info_df = pd.DataFrame(data = all_columns, 
             columns = ["Column_name", "Column_type", "Null", "Key", "Default", "Extra", "Table"])
# key, null and extra are all the same value, so I'm removing them
table_info_df.drop(columns = ['Null', "Key", "Extra", "Default"], inplace=True)

### Adding number of unique values to df

In [14]:
num_unique_values_list = []
for idx in range(len(table_info_df)):
    col_name = table_info_df['Column_name'][idx]
    table_name = table_info_df['Table'][idx]
    query = f"""
            SELECT COUNT(DISTINCT({col_name}))
            FROM {table_name}
    """
    query_result = run_query(query)
    num_unique_values_list.append(query_result)

In [15]:
# flattening the num_unique_values_list
flattened_num_unique_values = [num_unique_values_list[x][0][0] for x in range(len(num_unique_values_list))]
table_info_df['Number_of_unique_values'] = flattened_num_unique_values

In [16]:
# previewing table
table_info_df[0:3]

,Column_name,Column_type,Table,Number_of_unique_values
0,game_id,int(11),game,5718
1,season,int(11),game,10
2,type,varchar(1),game,3


### Saving DF

In [17]:
# table_info_df.to_csv("../data/table_info_df.csv")

## Listing all events

In [18]:
# what events are possible
events_query = """
        SELECT DISTINCT(event)
        FROM game_plays
"""
all_events = run_query(events_query)

### Saving All events as a list

In [19]:
with open('../data/all_event_types', 'wb') as f:
    pickle.dump(all_events, f)

<IPython.core.display.Javascript object>